In [6]:
from multiprocessing import Pool
from functools import partial
import time

In [7]:
nucleic_codes = list("ACGTURYKMSWBDHVN-")
amino_codes = list("ABCDEFGHIKLMNPQRSTUVWYZX*-")
digits = list("0123456789")

def test_fasta(filename, isnucleic = False):
    f = open(filename, 'r')
    line = f.readline().strip() #https://www.guru99.com/python-file-readline.html
    while line:
        if line[0] == '>':
            if line[1] == ' ':
                return False
            #adjust based on type
        else:
            if isnucleic:
                for char in line:
                    if char not in digits and char.upper() not in nucleic_codes:
                        print(f"Error at: {line}")
                        return False
            else:
                for char in line:
                    if char not in digits and char.upper() not in amino_codes:
                        print(f"Error at: {line}")
                        return False
        line = f.readline().strip()
    f.close()
    return True #all lines are correct

def parse_fasta(filename):
    if (not test_fasta(filename, False)):
        print("Invalid file")
    f = open(filename, 'r')
    line = f.readline().strip() 
    desc = ""
    seq = ""
    while line:
        if line[0] == '>':
            if desc != "":
                yield (desc, seq)
                seq = ""
            desc = line[1:]
        else:
            seq += line
        line = f.readline().strip()
    yield (desc, seq)
    f.close()

In [15]:
codon_table = { "UUU": "F", "UUC": "F", "UUA":"L", "UUG":"L", "CUU": "L", "CUC": "L", "CUA":"L", "CUG":"L",
              "AUU": "I", "AUC": "I", "AUA":"I", "AUG":"M", "GUU": "V", "GUC": "V", "GUA":"V", "GUG":"V",
              "UCU":"S", "UCC":"S", "UCA":"S","UCG":"S","CCU":"P", "CCC":"P", "CCA":"P","CCG":"P",
              "ACU":"U", "ACC":"U", "ACA":"U","ACG":"U","GCU":"A", "GCC":"A", "GCA":"A","GCG":"A",
              "UAU":"Y","UAC":"Y","UAA":"Z","UAG":"Z","CAU":"H","CAC":"H","CAA":"Q","CAG":"Q",
              "AAU":"N","AAC":"N", "AAA":"K", "AAG":"K","GAU":"D","GAC":"D","GAA":"E","GAG":"E",
              "UGU":"C","UGC":"C","UGA":"Z","UGG":"W","CGU":"R","CGC":"R","CGA":"R","CGG":"R",
              "AGU":"S","AGC":"S","AGA":"R","AGG":"R","GGU":"G","GGC":"G","GGA":"G","GGG":"G"}

def singlereadingframe(offset, rnastring):
    frames = []
    starttime = time.perf_counter()
    if (offset < 0):
        rnastring = rnastring[::-1]
    loc = abs(offset) - 1
    code = ""
    start = -1
    end = -1
    count = 0
    length = 0
    uncoded = ""
    while (loc + 2 < len(rnastring) and count < 2):
        codon = rnastring[loc:loc + 3]
        output = codon_table[codon]
        if (output == "M" and count == 0):
            count += 1
            start = loc
            length = 1
        if (output == "Z" and count == 1):
            count += 1
            end = loc
        if (count == 1):
            code += output
            uncoded += codon
            length += 1
        loc += 3
        if (count == 2):
            if (length >= 32):
                #print(f"* {offset} | {start} | {end} | {len(code)} | {code} \n")
                frames.append(uncoded)
            count = 0
            code = ""
            uncoded = ""
            length = 0
    endtime = time.perf_counter()
    print(f"The subprogram took {endtime - starttime} seconds.")
    return(frames)

In [16]:
nucleic_codes = list("ACGTURYKMSWBDHVN-")
amino_codes = list("ABCDEFGHIKLMNPQRSTUVWYZX*-")
digits = list("0123456789")

def test_fasta(filename, isnucleic = False):
    f = open(filename, 'r')
    line = f.readline().strip() #https://www.guru99.com/python-file-readline.html
    while line:
        if line[0] == '>':
            if line[1] == ' ':
                return False
            #adjust based on type
        else:
            if isnucleic:
                for char in line:
                    if char not in digits and char.upper() not in nucleic_codes:
                        print(f"Error at: {line}")
                        return False
            else:
                for char in line:
                    if char not in digits and char.upper() not in amino_codes:
                        print(f"Error at: {line}")
                        return False
        line = f.readline().strip()
    f.close()
    return True #all lines are correct

def parse_fasta(filename):
    if (not test_fasta(filename, False)):
        print("Invalid file")
    f = open(filename, 'r')
    line = f.readline().strip() 
    desc = ""
    seq = ""
    while line:
        if line[0] == '>':
            if desc != "":
                yield (desc, seq)
                seq = ""
            desc = line[1:]
        else:
            seq += line
        line = f.readline().strip()
    yield (desc, seq)
    f.close()
valid = ['T', 'A', 'C', 'G']
def dnatorna(dnastring):
    rna = ""
    for letter in dnastring:
        if letter in valid:
            if (letter == 'T'):
                rna += "U"
            else:
                rna += letter
    return rna

In [17]:
types = ["C", "G", "T", "A"]

def hascpgisland(text):  
    seq = ""
    textlen = length(text)
    end = 0
    while (len(seq) < 200 and end < textlen):
        nextchar = text[end].upper()
        if (nextchar in types):
            seq += nextchar
        end += 1
    ccount = seq.count('C')
    gcount = seq.count('G')
    observed = seq.count('CG')
    total = ccount + gcount + seq.count('T') + seq.count('A')
    if ((ccount + gcount) / total > 0.5):
        expected = ccount * gcount / total
        if (expected != 0 and observed / expected > 0.6):
            return True

    while end < textlen:
        nextchar = text[end].upper()
        if (nextchar in types):
            if (seq[0] == 'C'):
                ccount += -1
                if (seq[1] == 'G'):
                    observed += -1
            elif (seq[0] == 'G'):
                gcount += -1
            seq = seq[1:]
            seq += nextchar 
            if (nextchar == 'C'):
                ccount += 1
            elif (nextchar == 'G'):
                gcount += 1
                if (seq[-2] == 'C'):
                    observed += 1
            if ((ccount + gcount) / total > 0.5):
                expected = ccount * gcount / total
                if (expected != 0  and observed / expected > 0.6):
                    return True
        end += 1
    return False

In [19]:
start = time.perf_counter()
#file_path = "C:/Users/ewu15/jup/Scientific Computing/week 1/data/data/dogma.fasta.txt"
file_path = "C:/Users/ewu15/jup/Scientific Computing/week 1/data/data/sars-cov2-genome-fasta-filtered.txt"
#file_path = "C:/Users/ewu15/jup/Scientific Computing/week 2/sequence.fasta"
out = list(parse_fasta(file_path))
dnatext = out[0][1]
rnatext = dnatorna(dnatext)

func = partial(singlereadingframe, rnastring = rnatext)
offsets = [-3,-2,-1,1,2,3]
framelist = []
for i in offsets:
    frames = func(i)
    for frame in frames:
        if hascpgframe():
            frames.append(framelist)

end = time.perf_counter()
print(f"The program took {end - start} seconds.")

The subprogram took 0.009819700000207376 seconds.
AUGACAACAUGGAGAUUGUGUGAGAACCAUCACUAUUUCGAGUGUUCAUCGCUCACAAUAGUCACGGUUCUUUUCUUAUUAAAAGUACAAGCAAAU
AUGCUUCUACAGGUGCUUCCUAGUGUCGAGGCUAAUGCUCAAGUGAAAGAUCUUCGCCAGACCAGUCUUAUCACGGUACCUCACCGUGCAACUCUUCUUACAAUCAAAGACCUAACUUACUGGUGUACCUUGCGCAUGCGCGUUUGUCAGACUUUCUUCGUUACUUCAUCGACUCGGUGUAGUUCGGAUGUUCUGUUCGGUAACGCUAUCGUUAAGGUGGCCACUAGGUUAAAUAAGACAUUUGUCGUCGUUCGUGUUUUGUUCGAUUUCAAUGACCGGUAUUGUCGGUCUCCUUUUAAU
AUGUUUGUUGUUGUCGUUCAACGUUUGUUUCACUUGUGGGAACCUCUCACGAUCAACGGUAGAGAAAAACUCCCAAUACUAAAACCUUCGCGAGACUUUUUGUCGUUCUUCACGUUGCGGUUGUUAUUCGGUAGGCUUUCCCUCACUCCGAACAUAGCCAUAGCAACGUCAUCGCGCUUGUUU
AUGGUACCGGUAAAAUAUAUGACGAGUAUGAAAGGUUCAAGAACCUCUAGCUACUCUCUAAGUAAAUUUAAGAACCGUUGGAGUAACUCCGCCAGU
AUGUCCAUUAAUAUUAAUGGUGGUUGGAAUCUUAGUUCUAACAAUCUUAAGGUUCGAUAUUGCGUCGGACAUUUUAGUAGACCAUUAAAUAUUAAUAUUAGUCGU
AUGUGUGAAUUAUCGAAAUCCCAAUGGUUACAGCACUUCUUGACCCUUAAACAGACGAUUAUUACGUCGAAAUUGUUGGUACACAACUUGGAAAGAUGUUCGGCG
AUGUCAUCGUUAUGGUAUUCUGUCAAAUUUACAGAGGAGUCAUCGAAACUCGCAAAGAC

In [ ]:
#Must begin from a start codon done
#ORF must contain at least 96 bp (32 amino acids) done
#ORF must occur in a CpG island
#CpG should appear in first third of the sequence 
#can't really tell if orfs are inside cpg or otherway around
#above kind of assumes that cpgs are supposed to be within 